# Describe a DataFrame

Your data processing in Azure Databricks is accomplished by defining Dataframes to read and process the Data.

This notebook will introduce how to read your data using Azure Databricks Dataframes.

#Introduction

** Data Source **
* One hour of Pagecounts from the English Wikimedia projects captured August 5, 2016, at 12:00 PM UTC.
* Size on Disk: ~23 MB
* Type: Compressed Parquet File
* More Info: <a href="https://dumps.wikimedia.org/other/pagecounts-raw" target="_blank">Page view statistics for Wikimedia projects</a>

**Technical Accomplishments:**
* Develop familiarity with the `DataFrame` APIs
* Introduce the classes...
  * `SparkSession`
  * `DataFrame` (aka `Dataset[Row]`)
* Introduce the actions...
  * `count()`

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Getting Started

Run the following cell to configure our "classroom."

In [0]:
%run "./Includes/Classroom-Setup"

Initialized classroom variables & functions...

Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

In [0]:
%run "./Includes/Utility-Methods"

Created user-specific database

Using the database arpit_mittal_team_telstra_com_db .

All done!

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) **The Data Source**

* In this notebook, we will be using a compressed parquet "file" called **pagecounts** (~23 MB file from Wikipedia)
* We will explore the data and develop an understanding of it as we progress.
* You can read more about this dataset here: <a href="https://dumps.wikimedia.org/other/pagecounts-raw/" target="_blank">Page view statistics for Wikimedia projects</a>.

We can use **dbutils.fs.ls()** to view our data on the DBFS.

In [0]:
(source, sasEntity, sasToken) = getAzureDataSource()

spark.conf.set(sasEntity, sasToken)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1330340542039256> in <module> 
 ----> 1 ( source , sasEntity , sasToken ) = getAzureDataSource ( ) 
 2 
 3 spark . conf . set ( sasEntity , sasToken ) 

 <command-1330340542039235> in getAzureDataSource () 
 35 
 36 def getAzureDataSource ( ) : 
 ---> 37 datasource = spark . conf . get ( "com.databricks.training.azure.datasource" ) . split ( "\t" ) 
 38 source = datasource [ 0 ] 
 39 sasEntity = datasource [ 1 ] 

 /databricks/spark/python/pyspark/sql/conf.py in get (self, key, default) 
 49 self . _checkType ( key , "key" ) 
 50 if default is _NoValue : 
 ---> 51 return self . _jconf . get ( key ) 
 52 else : 
 53 if default is not None : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o255.get.
: java.util.NoSuchElementException: com.databricks.training.azure.datasource
	at org.apache.spark.sql.internal.SQLConf.$anonfun$getConfString$3(SQLConf.scala:3418)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.internal.SQLConf.getConfString(SQLConf.scala:3418)
	at org.apache.spark.sql.RuntimeConfig.get(RuntimeConfig.scala:74)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [0]:
# hard-coding value of source path since above cell's "getAzureDataSource()" function is throwing error.
source = "/mnt/training"

path = source + "/wikipedia/pagecounts/staging_parquet_en_only_clean/"
files = dbutils.fs.ls(path)
display(files)

path,name,size
dbfs:/mnt/training/wikipedia/pagecounts/staging_parquet_en_only_clean/_SUCCESS,_SUCCESS,0
dbfs:/mnt/training/wikipedia/pagecounts/staging_parquet_en_only_clean/_committed_6241970109963426653,_committed_6241970109963426653,760
dbfs:/mnt/training/wikipedia/pagecounts/staging_parquet_en_only_clean/_started_6241970109963426653,_started_6241970109963426653,0
dbfs:/mnt/training/wikipedia/pagecounts/staging_parquet_en_only_clean/part-00000-tid-6241970109963426653-cd9cd6ee-cb10-4da2-82b3-ea25a8369cbf-0-c000.gz.parquet,part-00000-tid-6241970109963426653-cd9cd6ee-cb10-4da2-82b3-ea25a8369cbf-0-c000.gz.parquet,2996913
dbfs:/mnt/training/wikipedia/pagecounts/staging_parquet_en_only_clean/part-00001-tid-6241970109963426653-cd9cd6ee-cb10-4da2-82b3-ea25a8369cbf-0-c000.gz.parquet,part-00001-tid-6241970109963426653-cd9cd6ee-cb10-4da2-82b3-ea25a8369cbf-0-c000.gz.parquet,2994285
dbfs:/mnt/training/wikipedia/pagecounts/staging_parquet_en_only_clean/part-00002-tid-6241970109963426653-cd9cd6ee-cb10-4da2-82b3-ea25a8369cbf-0-c000.gz.parquet,part-00002-tid-6241970109963426653-cd9cd6ee-cb10-4da2-82b3-ea25a8369cbf-0-c000.gz.parquet,2994196
dbfs:/mnt/training/wikipedia/pagecounts/staging_parquet_en_only_clean/part-00003-tid-6241970109963426653-cd9cd6ee-cb10-4da2-82b3-ea25a8369cbf-0-c000.gz.parquet,part-00003-tid-6241970109963426653-cd9cd6ee-cb10-4da2-82b3-ea25a8369cbf-0-c000.gz.parquet,2992431
dbfs:/mnt/training/wikipedia/pagecounts/staging_parquet_en_only_clean/part-00004-tid-6241970109963426653-cd9cd6ee-cb10-4da2-82b3-ea25a8369cbf-0-c000.gz.parquet,part-00004-tid-6241970109963426653-cd9cd6ee-cb10-4da2-82b3-ea25a8369cbf-0-c000.gz.parquet,2990093
dbfs:/mnt/training/wikipedia/pagecounts/staging_parquet_en_only_clean/part-00005-tid-6241970109963426653-cd9cd6ee-cb10-4da2-82b3-ea25a8369cbf-0-c000.gz.parquet,part-00005-tid-6241970109963426653-cd9cd6ee-cb10-4da2-82b3-ea25a8369cbf-0-c000.gz.parquet,2989931
dbfs:/mnt/training/wikipedia/pagecounts/staging_parquet_en_only_clean/part-00006-tid-6241970109963426653-cd9cd6ee-cb10-4da2-82b3-ea25a8369cbf-0-c000.gz.parquet,part-00006-tid-6241970109963426653-cd9cd6ee-cb10-4da2-82b3-ea25a8369cbf-0-c000.gz.parquet,2989314


As we can see from the files listed above, this data is stored in <a href="https://parquet.apache.org" target="_blank">Parquet</a> files which can be read in a single command, the result of which will be a `DataFrame`.

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Create a DataFrame
* We can read the Parquet files into a `DataFrame`.
* We'll start with the object **spark**, an instance of `SparkSession` and the entry point to Spark 2.0 applications.
* From there we can access the `read` object which gives us an instance of `DataFrameReader`.

In [0]:
parquetDir = source + "/wikipedia/pagecounts/staging_parquet_en_only_clean/"

In [0]:
pagecountsEnAllDF = (spark  # Our SparkSession & Entry Point
  .read                     # Our DataFrameReader
  .parquet(parquetDir)      # Returns an instance of DataFrame
)
print(pagecountsEnAllDF)    # Python hack to see the data type

DataFrame[project: string, article: string, requests: int, bytes_served: bigint]

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) count()

If you look at the API docs, `count()` is described like this:
> Returns the number of rows in the Dataset.

`count()` will trigger a job to process the request and return a value.

We can now count all records in our `DataFrame` like this:

In [0]:
total = pagecountsEnAllDF.count()

print("Record Count: {0:,}".format( total ))

Record Count: 2,345,943

That tells us that there are around 2 million rows in the `DataFrame`.

## Next steps

Start the next lesson, [Use common DataFrame methods]($./2.Use-common-dataframe-methods)